In [ ]:
import numpy as np
import skimage
import skimage.filters
import nd2reader
import matplotlib.pyplot as plt

In [ ]:
nd2 = nd2reader.ND2Reader("/n/scratch2/jqs1/fidelity/all/180405_txnerr.nd2")

In [ ]:
img = np.asarray(nd2.get_frame_2D(t=0, c=2, v=50))

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(img)

In [ ]:
bin_img = img > skimage.filters.threshold_otsu(img)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(bin_img)

In [ ]:
%%time
h, theta, d = skimage.transform.hough_line(bin_img)

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(h, aspect=0.02)

In [ ]:
# FROM: https://alyssaq.github.io/2014/understanding-hough-transform/
def hough_line(img, theta=None):
    # Rho and Theta ranges
    if theta is None:
        theta = np.linspace(-np.pi / 2, np.pi / 2, 180)
    width, height = img.shape
    diag_len = int(np.ceil(np.sqrt(width * width + height * height)))  # max_dist
    rhos = np.linspace(-diag_len, diag_len, diag_len * 2)

    # Cache some resuable values
    cos_t = np.cos(theta)
    sin_t = np.sin(theta)
    num_thetas = len(theta)

    # Hough accumulator array of theta vs rho
    accumulator = np.zeros((2 * diag_len, num_thetas), dtype=np.uint64)
    y_idxs, x_idxs = np.nonzero(img)  # (row, col) indexes to edges

    # Vote in the hough accumulator
    for i in range(len(x_idxs)):
        x = x_idxs[i]
        y = y_idxs[i]
        rho = np.round(x * cos_t + y * sin_t).astype(np.int) + diag_len
        accumulator[rho, np.arange(num_thetas)] += 1

    return accumulator, theta, rhos

In [ ]:
plt.imshow(bin_img[100:500, :])

In [ ]:
%%time
h2, theta2, d2 = hough_line(bin_img)

In [ ]:
h.shape

In [ ]:
h2.shape

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(h2, aspect=0.02)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(h - h2, aspect=0.02)

In [ ]:
bin_img.dtype

In [ ]:
np.iinfo(bin_img.dtype)

In [ ]:
def _accumulator_dtype(dtype):
    if np.issubdtype(dtype, np.bool_):
        return np.uint64
    elif np.issubdtype(dtype, np.signedinteger):
        return np.int64
    elif np.issubdtype(dtype, np.unsignedinteger):
        return np.uint64
    elif np.issubdtype(dtype, np.floating):
        return np.float64
    elif np.issubdtype(dtype, np.complex):
        return np.complex128
    else:
        return NotImplementedError


# FROM: https://alyssaq.github.io/2014/understanding-hough-transform/
def hough_line_intensity(img, theta=None):
    # Rho and Theta ranges
    if theta is None:
        theta = np.linspace(-np.pi / 2, np.pi / 2, 180)
    width, height = img.shape
    diag_len = int(np.ceil(np.sqrt(width * width + height * height)))
    rhos = np.linspace(-diag_len, diag_len, diag_len * 2)

    # Cache some resuable values
    cos_t = np.cos(theta)
    sin_t = np.sin(theta)
    num_thetas = len(theta)

    # Hough accumulator array of theta vs rho
    accumulator = np.zeros(
        (2 * diag_len, num_thetas), dtype=_accumulator_dtype(img.dtype)
    )  # TODO: dtype

    # Vote in the hough accumulator
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if img[i, j] == 0:  # optimization for boolean input images
                continue
            rho = np.round(i * sin_t + j * cos_t).astype(np.int_) + diag_len
            accumulator[rho, np.arange(num_thetas)] += img[i, j]

    return accumulator, theta, rhos

In [ ]:
%%time
h3, theta3, d3 = hough_line_intensity(bin_img)

In [ ]:
%%time
h4, theta4, d4 = hough_line_intensity(img)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(h[3000:3200, 80:100], aspect=0.1)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(h4[3000:3200, 80:100], aspect=0.1)

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(h[3000:3200, 90] / h.max())
plt.plot(h4[3000:3200, 90] / h4.max())

In [ ]:
h3

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow((h3.astype(np.int64) - h2), aspect=0.02)

In [ ]:
(h3.astype(np.int64) - h).max()

In [ ]:
h.max()

In [ ]:
(h / 902 - h3 / h3.max()).max()

In [ ]:
h3.max()

In [ ]:
plt.scatter(h.flat, h3.flat)